In [1]:
import yfinance as yf

#define the ticker symbol for NIFTY 50
ticker = "^NSEI"  #NIFTY 50 index symbol on Yahoo Finance

#fetch the data from October 26, 2019 to October 26, 2024
nifty_data = yf.download(ticker, start="2019-10-26", end="2024-10-26", interval="1d")

#save the data to a CSV file
nifty_data.to_csv("nifty_50_data.csv")

print("Data saved to nifty_50_data.csv")

[*********************100%***********************]  1 of 1 completed

Data saved to nifty_50_data.csv


In [2]:
import pandas as pd

# Load the dataset
file_path = r"C:\Users\Anshul\OneDrive\Music\nifty_50_data.csv"
nifty_data = pd.read_csv(file_path)

# Remove the first two rows with header information and reset the index
nifty_data_cleaned = nifty_data.iloc[2:].reset_index(drop=True)

# Rename columns to ensure they are properly named
nifty_data_cleaned.columns = ["Date", "Adj Close", "Close", "High", "Low", "Open", "Volume"]

# Convert 'Date' column to datetime format
nifty_data_cleaned["Date"] = pd.to_datetime(nifty_data_cleaned["Date"])

# Convert numeric columns to float for analysis
numeric_columns = ["Adj Close", "Close", "High", "Low", "Open", "Volume"]
nifty_data_cleaned[numeric_columns] = nifty_data_cleaned[numeric_columns].astype(float)

# Display the cleaned dataset
nifty_data_cleaned.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2019-10-29 00:00:00+00:00,11786.849609,11786.849609,11809.400391,11627.349609,11643.950195,951700.0
1,2019-10-30 00:00:00+00:00,11844.099609,11844.099609,11883.950195,11784.450195,11883.900391,725200.0
2,2019-10-31 00:00:00+00:00,11877.450195,11877.450195,11945.000000,11855.099609,11890.450195,1414800.0
3,2019-11-01 00:00:00+00:00,11890.599609,11890.599609,11918.299805,11843.349609,11886.599609,855300.0
4,2019-11-04 00:00:00+00:00,11941.299805,11941.299805,11989.150391,11905.349609,11928.900391,823100.0


In [3]:
# Save the cleaned dataset to a new CSV file
nifty_data_cleaned.to_csv("cleaned_nifty_50.csv", index=False)

In [11]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch data from GDELT API with error handling for empty responses
def fetch_from_gdelt(query, start_date, end_date):
    base_url = "https://api.gdeltproject.org/api/v2/doc/doc?"
    url = f"{base_url}query={query}&startdatetime={start_date}&enddatetime={end_date}&format=json"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            try:
                json_data = response.json()
                if 'articles' in json_data:
                    return json_data
                else:
                    print(f"No articles found for query: {query}")
                    return None
            except ValueError:
                print(f"Invalid or empty JSON response for query: {query}")
                return None
        else:
            print(f"GDELT API failed for query: {query} with status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed for query: {query}. Error: {e}")
        return None

# Function to generate monthly time intervals over a 5-year period
def generate_monthly_intervals(start_date, end_date):
    intervals = []
    current_date = start_date
    while current_date < end_date:
        next_date = current_date + timedelta(days=30)  # Approximate month duration
        intervals.append((current_date.strftime("%Y%m%d%H%M%S"), next_date.strftime("%Y%m%d%H%M%S")))
        current_date = next_date
    return intervals

# Function to handle multiple queries and monthly time intervals
def fetch_news_data():
    final_data = []

    # Broader, simplified queries for higher likelihood of valid responses
    queries = [
        "Indian stock market", "NSE trends", "NIFTY trends India",
        "interest rates India", "RBI policy India", "India inflation",
        "consumer price index India", "GDP growth India", 
        "India unemployment", "India industrial production",
        "crude oil prices India", "technology sector India", 
        "banking sector India", "financial services India",
        "healthcare India", "pharma stocks India",
        "foreign investment India", "domestic investment India",
        "NSE earnings", "corporate profits India"
    ]

    # Define the start and end date (5-year period)
    start_date = datetime(2019, 10, 1, 0, 0, 0)
    end_date = datetime(2024, 10, 1, 0, 0, 0)

    # Generate monthly intervals
    time_intervals = generate_monthly_intervals(start_date, end_date)

    # Loop through each query and monthly time period
    for query in queries:
        for start, end in time_intervals:
            gdelt_data = fetch_from_gdelt(query, start, end)
            if gdelt_data and 'articles' in gdelt_data:
                for article in gdelt_data['articles']:
                    final_data.append({
                        'Date': article.get('seendate', 'No Date'),
                        'Title': article.get('title', 'No Title'),
                        'URL': article.get('url', 'No URL')
                    })

    return final_data

# Run the news fetch and save data to CSV
news_data = fetch_news_data()
df = pd.DataFrame(news_data)
df.drop_duplicates(inplace=True)  # Remove duplicate articles
df.to_csv('nifty50_gdelt_news.csv', index=False)
print("Data has been saved to nifty50_gdelt_news.csv")


GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API failed for query: India inflation with status code: 429
GDELT API 

In [12]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Path to our ChromeDriver executable
chrome_driver_path =r"C:\Users\Anshul\OneDrive\Videos\chromedriver-win64\chromedriver-win64\chromedriver.exe"  

# Function to extract text from a URL using Selenium
def extract_text_from_url(url):
    try:
        # Set up Selenium WebDriver
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run headless to avoid opening a browser window
        chrome_service = Service(executable_path=chrome_driver_path)
        driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

        # Load the webpage
        driver.get(url)
        time.sleep(3)  # Wait for the page to load fully

        # Extract text content from the webpage using Selenium
        paragraphs = driver.find_elements(By.TAG_NAME, "p")  # Find all <p> elements
        text_content = ' '.join([para.text for para in paragraphs])  # Get the text from each paragraph

        driver.quit()  # Close the browser
        return text_content

    except Exception as e:
        print(f"Failed to extract text from {url}: {e}")
        return None

# Function to read the CSV and extract content for each URL
def extract_content_from_csv(input_csv, output_csv):
    # Load the data
    df = pd.read_csv(input_csv)

    # Add a new column for extracted content
    df['Content'] = df['URL'].apply(lambda url: extract_text_from_url(url) if pd.notnull(url) else 'No URL')

    # Save the updated DataFrame with content to a new CSV
    df.to_csv(output_csv, index=False)
    print(f"Data with extracted content has been saved to {output_csv}")

# Replace 'input.csv' with our actual CSV file containing URLs
input_csv = r"C:\Users\Anshul\OneDrive\Music\nifty50_gdelt_news.csv"  
output_csv = 'nifty50_gdelt_news_extracted.csv'  # Output file with extracted content

# Run the extraction
extract_content_from_csv(input_csv, output_csv)

Failed to extract text from https://www.aninews.in/news/business/bse-signs-pact-with-ihs-markit-to-develop-new-bond-valuation-service20191217170141/: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=130.0.6723.70)
Stacktrace:
	GetHandleVerifier [0x00007FF7ED47B095+29557]
	(No symbol) [0x00007FF7ED3EFA50]
	(No symbol) [0x00007FF7ED2AB56A]
	(No symbol) [0x00007FF7ED2A356D]
	(No symbol) [0x00007FF7ED294459]
	(No symbol) [0x00007FF7ED296202]
	(No symbol) [0x00007FF7ED29471F]
	(No symbol) [0x00007FF7ED293FAB]
	(No symbol) [0x00007FF7ED293EEA]
	(No symbol) [0x00007FF7ED291D65]
	(No symbol) [0x00007FF7ED2923DC]
	(No symbol) [0x00007FF7ED2AE6A1]
	(No symbol) [0x00007FF7ED34933E]
	(No symbol) [0x00007FF7ED32718A]
	(No symbol) [0x00007FF7ED34851C]
	(No symbol) [0x00007FF7ED326F33]
	(No symbol) [0x00007FF7ED2F116F]
	(No symbol) [0x00007FF7ED2F22D1]
	GetHandleVerifier [0x00007FF7ED7AC96D+3378253]
	GetHandleVerifier [0x00007FF7ED7F8497+3688311]
	GetHandleVerifier [0x000